In [ ]:
#!pip install -U tensorflow==2.0.0

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from xgboost import DMatrix

print('Tensorflow ', tf.__version__)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#%matplotlib inline

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print('Running on TPU :', tpu.master())
    
except ValueError:
    tpu = None
    
if tpu:
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
else:
    strategy = tf.distribute.get_strategy()
    
print('Replicas: ', strategy.num_replicas_in_sync)

In [ ]:
train_data = pd.read_json('/kaggle/input/stanford-covid-vaccine/train.json', lines=True)
test_data = pd.read_json('/kaggle/input/stanford-covid-vaccine/test.json', lines=True)

In [ ]:
submit = pd.read_csv('/kaggle/input/stanford-covid-vaccine/sample_submission.csv')

In [ ]:
submit.head()

In [ ]:
train_data.head()

In [ ]:
print(train_data.shape)

In [ ]:
print(train_data.columns)

# Feature Extraction

In [ ]:
def in_out_shape(data, input_shape=107, output_shape=68, contra=True):
    if type(data) != 'list':
        d = list(data)
    else:
        d = data
        
    if (input_shape == 107 and output_shape == 107 and contra == True)\
        or (input_shape == 68 and output_shape == 107):
        
        d = d[:68]
        d.extend(d[:39])
        return d
    
    elif (input_shape == 107 and output_shape == 68)\
        or (input_shape == 68 and output_shape == 68):
        
        d = d[:68]
        return d
    
    elif input_shape == 107 and output_shape == 107 and contra == False:
        d = d[:107]
        return d
    elif input_shape == 130 and output_shape == 130 and contra == False:
        d = d[:130]
        return d

In [ ]:
def f1_ls_to_np(data, ishp, oshp, c):
    d = in_out_shape(data, ishp, oshp, c)
    ls = []
    for i in range(len(d)):
        if d[i] == 'A':
            ls.append([1, 0, 0, 0])
        elif d[i] == 'U':
            ls.append([0, 1, 0, 0])
        elif d[i] == 'G':
            ls.append([0, 0, 1, 0])
        else:
            ls.append([0, 0, 0, 1])
    ls = np.array(ls)
    return ls

def bases(data, i_shp = 107, o_shp = 107, c = True):
    obj = np.expand_dims(f1_ls_to_np(data[0], i_shp, o_shp, c), axis=0)
    for i in range(1, len(data)):
        obj = np.concatenate((obj, np.expand_dims(f1_ls_to_np(data[i], i_shp, o_shp, c), axis=0)))
    return obj

In [ ]:
def struct_1_hot(data, index, isp, test):
    if test == True and isp == 107:
        bases = ts_df_107.sequence[index]
    elif test == True and isp == 130:
        bases = ts_df_130.sequence[index]
    else:
        bases = train_data.sequence[index]
    idxs = []
    struct_1_hot = [i for i in range(0, len(data))]
    for idx, item in enumerate(data):
        if item == '.':
            struct_1_hot[idx] = [0, 0, 0, 0, 0, 0]
        elif item == '(':
            idxs.append(idx)
        elif item == ')':
            first = idxs.pop()
            second = idx
            if bases[first] == 'U':
                if bases[second] == 'G': # U-G
                    struct_1_hot[first] = [1, 0, 0, 0, 0, 0]
                    struct_1_hot[second] = [1, 0, 0, 0, 0, 0]
                else: # U-A
                    struct_1_hot[first] = [0, 0, 0, 1, 0, 0]
                    struct_1_hot[second] = [0, 0, 0, 1, 0, 0]
            elif bases[first] == 'G':
                if bases[second] == 'U': # G-U
                    struct_1_hot[first] = [0, 1, 0, 0, 0, 0]
                    struct_1_hot[second] = [0, 1, 0, 0, 0, 0]
                else: # G-C
                    struct_1_hot[first] = [0, 0, 0, 0, 1, 0]
                    struct_1_hot[second] = [0, 0, 0, 0, 1, 0]
            elif bases[first] == 'A': # A-U
                struct_1_hot[first] = [0, 0, 1, 0, 0, 0]
                struct_1_hot[second] = [0, 0, 1, 0, 0, 0]
            else: #C-G
                struct_1_hot[first] = [0, 0, 0, 0, 0, 1]
                struct_1_hot[second] = [0, 0, 0, 0, 0, 1]
    return struct_1_hot

def f2_ls_to_np(data, index, isp, osp, c, t):
    d = list(data)
    d = struct_1_hot(d, index, isp, t)
    d = in_out_shape(d, isp, osp, c)
    d = np.array(d)
    return d

def structs(data, isp = 107, osp = 107, c = True, t = False):
    obj = np.expand_dims(f2_ls_to_np(data[0], 0, isp, osp, c, t), axis=0)
    for i in range(1, len(data)):
        obj = np.concatenate((obj, np.expand_dims(f2_ls_to_np(data[i], i, isp, osp, c, t), axis=0)))
    return obj

In [ ]:
def f3_ls_to_np(data, isp, osp, c):
    d = in_out_shape(data, isp, osp, c)
    ls = []
    for i in range(len(d)):
        if d[i] == 'E':#Dangling End
            ls.append([1, 0, 0, 0, 0, 0])
        elif d[i] == 'S':#Paired Stem
            ls.append([0, 1, 0, 0, 0, 0])
        elif d[i] == 'H':#Hairpin
            ls.append([0, 0, 1, 0, 0, 0])
        elif d[i] == 'B':#Bulge
            ls.append([0, 0, 0, 1, 0, 0])
        elif d[i] == 'x':#eXternal Loop
            ls.append([0, 0, 0, 0, 1, 0])
        else:# 'I' #Internal Loop
            ls.append([0, 0, 0, 0, 0, 1])
    ls = np.array(ls)
    return ls

def loop_types(data, isp = 107, osp = 107, c = True):
    obj = np.expand_dims(f3_ls_to_np(data[0], isp, osp, c), axis=0)
    for i in range(1, len(data)):
        obj = np.concatenate((obj, np.expand_dims(f3_ls_to_np(data[i], isp, osp, c), axis=0)))
    return obj

### Feature-1 // bases

In [ ]:
# A = [1, 0, 0, 0]
# U = [0, 1, 0, 0]
# G = [0, 0, 1, 0]
# C = [0, 0, 0, 1]

f1 = train_data.sequence

bases_68_107 = bases(f1, 107, 107, True)

bases_68 = bases(f1, 68, 68, False)

bases_107 = bases(f1, 107, 107, False)
print(bases_107.shape)

In [ ]:
print(bases_107)

### Feature-2 // structures

In [ ]:
# U-G = [1, 0, 0, 0, 0, 0]
# G-U = [0, 1, 0, 0, 0, 0]
# A-U = [0, 0, 1, 0, 0, 0]
# U-A = [0, 0, 0, 1, 0, 0]
# G-C = [0, 0, 0, 0, 1, 0]
# C-G = [0, 0, 0, 0, 0, 1]


f2 = train_data.structure

structs_68_107 = structs(f2, 107, 107, True)

structs_68 = structs(f2, 68, 68, False)

structs_107 = structs(f2, 107, 107, False)
print(structs_107.shape)

### Feature-3 // Loop type

In [ ]:
# E = [1, 0, 0, 0, 0, 0]
# S = [0, 1, 0, 0, 0, 0]
# H = [0, 0, 1, 0, 0, 0]
# B = [0, 0, 0, 1, 0, 0]
# X = [0, 0, 0, 0, 1, 0]
# I = [0, 0, 0, 0, 0, 1]

f3 = train_data.predicted_loop_type

loop_types_68_107 = loop_types(f3, 107, 107, True)

loop_types_68 = loop_types(f3, 68, 68, False)

loop_types_107 = loop_types(f3, 107, 107, False)
print(loop_types_107.shape)

# Target Extraction

In [ ]:
def t_ls_to_np(data, isp, osp, c):
    d = in_out_shape(data, isp, osp, c)
    d = np.array(d)
    d = np.expand_dims(d, axis=0)
    return d

def target(data, isp = 68, osp = 107, c = True):
    obj = t_ls_to_np(data[0], isp, osp, c)
    for i in range(1, len(data)):
        obj = np.concatenate((obj, t_ls_to_np(data[i], isp, osp, c)), axis=0)
    return obj

### Target-1 // reactivity

In [ ]:
t1 = train_data.reactivity

reactivity_68_107 = target(t1, 68, 107, True)

reactivity_68 = target(t1, 68, 68, False)
print(reactivity_68.shape)

In [ ]:
e1 = train_data.reactivity_error

e1_68 = target(e1, 68, 68, False)
e1_68.shape

### Target-2 // deg_Mg_pH10

In [ ]:
t2 = train_data.deg_Mg_pH10

deg_Mg_pH10_68_107 = target(t2, 68, 107, True)

deg_Mg_pH10_68 = target(t2, 68, 68, False)
print(deg_Mg_pH10_68.shape)

In [ ]:
e2 = train_data.deg_error_Mg_pH10

e2_68 = target(e2, 68, 68, False)
e2_68.shape

### Target-3 // deg_pH10

In [ ]:
t3 = train_data.deg_pH10

deg_pH10_68_107 = target(t3, 68, 107, True)

deg_pH10_68 = target(t3, 68, 68, False)
print(deg_pH10_68.shape)

In [ ]:
e3 = train_data.deg_error_pH10

e3_68 = target(e3, 68, 68, False)
e3_68.shape

### Target-4 // deg_Mg_50C

In [ ]:
t4 = train_data.deg_Mg_50C

deg_Mg_50C_68_107 = target(t4, 68, 107, True)

deg_Mg_50C_68 = target(t4, 68, 68, False)
print(deg_Mg_50C_68.shape)

In [ ]:
e4 = train_data.deg_error_Mg_50C

e4_68 = target(e4, 68, 68, False)
e4_68.shape

### Target-5 // deg_50C

In [ ]:
t5 = train_data.deg_50C

deg_50C_68_107 = target(t5, 68, 107, True)

deg_50C_68 = target(t5, 68, 68, False)
print(deg_50C_68.shape)

In [ ]:
e5 = train_data.deg_error_50C

e5_68 = target(e5, 68, 68, False)
e5_68.shape

# Test Data

In [ ]:
test_data.shape

In [ ]:
test_data.head()

In [ ]:
test_data.columns

In [ ]:
ts_df_107 = test_data[test_data['seq_length'] == 107][['id', 'sequence', 'structure', 'predicted_loop_type']].reset_index()
ts_df_107.head()

In [ ]:
ts_df_107.columns

In [ ]:
ts_df_130 = test_data[test_data['seq_length'] == 130][['id', 'sequence', 'structure', 'predicted_loop_type']].reset_index()
ts_df_130.head()

## Test Features

In [ ]:
id_col_107 = ts_df_107.id
id_col_107.head()

In [ ]:
id_col_130 = ts_df_130.id
id_col_130.head()

### Feature-1 // Base

In [ ]:
ts_f1_107 = ts_df_107.sequence

ts_f1_107 = bases(ts_f1_107, 107, 107, False)
print(ts_f1_107.shape)

In [ ]:
ts_f1_130 = ts_df_130.sequence

ts_f1_130 = bases(ts_f1_130, 130, 130, False)
ts_f1_130.shape

In [ ]:
print(ts_f1_130)

### Feature-2 // Structure

In [ ]:
ts_f2_107 = ts_df_107.structure

ts_f2_107 = structs(ts_f2_107, 107, 107, False, True)
print(ts_f2_107.shape)

In [ ]:
ts_f2_130 = ts_df_130.structure

ts_f2_130 = structs(ts_f2_130, 130, 130, False, True)
ts_f2_130.shape

### Feature-3 // Loop type

In [ ]:
ts_f3_107 = ts_df_107.predicted_loop_type

ts_f3_107 = loop_types(ts_f3_107, 107, 107, False)
print(ts_f3_107.shape)

In [ ]:
ts_f3_130 = ts_df_130.predicted_loop_type

ts_f3_130 = loop_types(ts_f3_130, 130, 130, False)
ts_f3_130.shape

# Single Base Variable Approach

In [ ]:
_end = np.expand_dims(np.array([0, 0, 0, 0]), axis=0)

def get_rows(i):
    rows = np.concatenate((_end, \
                           np.expand_dims(bases[i, 0, :], axis=0),\
                           np.expand_dims(bases[i, 1, :], axis=0),\
                           np.expand_dims(structs[i, 0, :], axis=0),\
                           np.expand_dims(loop_types[i, 0, :], axis=0)),\
                          axis=1)

    for j in range(1, seq_len - 1):
        row = np.concatenate((np.expand_dims(bases[i, j-1, :], axis=0), \
                              np.expand_dims(bases[i, j, :], axis=0),\
                              np.expand_dims(bases[i, j+1, :], axis=0),\
                              np.expand_dims(structs[i, j, :], axis=0),\
                              np.expand_dims(loop_types[i, j, :], axis=0)),\
                             axis=1)
        rows = np.concatenate((rows, row), axis=0)

    last = np.concatenate((np.expand_dims(bases[i, seq_len - 2, :], axis=0), \
                           np.expand_dims(bases[i, seq_len - 1, :], axis=0),\
                           _end,\
                           np.expand_dims(structs[i, seq_len - 1, :], axis=0),\
                           np.expand_dims(loop_types[i, seq_len - 1, :], axis=0)),\
                          axis=1)
    rows = np.concatenate((rows, last), axis=0)
    
    return rows
        

def samples():
    feats = get_rows(0)
    for i in range(1, len(bases)):
        feat = get_rows(i)
        feats = np.concatenate((feats, feat), axis=0)
    return feats

In [ ]:
seq_len = 68
bases = bases_68
structs = structs_68
loop_types = loop_types_68

features = samples()
features.shape

In [ ]:
features[:5]

### Test Features

In [ ]:
seq_len = 107
bases = ts_f1_107
structs = ts_f2_107
loop_types = ts_f3_107

ts_features_1 = samples()
ts_features_1.shape

In [ ]:
ts_features_1[:5]

In [ ]:
seq_len = 130
bases = ts_f1_130
structs = ts_f2_130
loop_types = ts_f3_130

ts_features_2 = samples()
ts_features_2.shape

In [ ]:
ts_features = np.concatenate((ts_features_1, ts_features_2), axis=0)
ts_features.shape

# Experimental

In [ ]:
f = features
print(f.shape)
feat_exp = []
for i in range(f.shape[0]):
    strng = ''
    for j in range(f.shape[1]):
        strng += str(f[i][j])
    feat_exp.append(strng)
print(feat_exp[:5], len(feat_exp))

In [ ]:
t1 = np.reshape(reactivity_68, (163200,))
t2 = np.reshape(deg_Mg_pH10_68, (163200,))
t3 = np.reshape(deg_pH10_68, (163200,))
t4 = np.reshape(deg_Mg_50C_68, (163200))
t5 = np.reshape(deg_50C_68, (163200,))

e1 = np.reshape(e1_68, (163200,))
e2 = np.reshape(e2_68, (163200,))
e3 = np.reshape(e3_68, (163200,))
e4 = np.reshape(e4_68, (163200,))
e5 = np.reshape(e5_68, (163200,))

exp_df = pd.DataFrame({
    'feat': feat_exp,
    't1': t1,
    'e1': e1,
    't2': t2,
    'e2': e2,
    't3': t3,
    'e3': e3,
    't4': t4,
    'e4': e4,
    't5': t5,
    'e5': e5
})
exp_df.head()

In [ ]:
exp_df.feat.nunique()

In [ ]:
f = ts_features
print(f.shape)
ts_feat_exp = []
for i in range(f.shape[0]):
    strng = ''
    for j in range(f.shape[1]):
        strng += str(f[i][j])
    ts_feat_exp.append(strng)
print(ts_feat_exp[:5], len(ts_feat_exp))

In [ ]:
ts_ft_df = pd.DataFrame({
    'ts_feat': ts_feat_exp
})
ts_ft_df.head()

## Filtration 

In [ ]:
def filtrate(t, e):
    t = np.reshape(t, (163200, 1))
    e = np.reshape(e, (163200, 1))
    
    e_fltr = np.squeeze(e <= 0.3)
    
    t = t[e_fltr]
    
    t_mean = np.mean(t)
    t_std = np.std(t)
    
    t_fltr = np.abs(t - t_mean) < 3 * t_std
    t_fltr = np.squeeze(t_fltr)
    
    t = t[t_fltr]
    
    return features[e_fltr][t_fltr], t

# Model

In [ ]:
def get_regressor():
    inp = Input(shape=(input_vec,))
    
    regr = Dense(50)(inp)
    regr = Dense(50)(regr)
    regr = Dense(50)(regr)
    outp = Dense(1)(regr)
    
    model = Model(inputs=inp, outputs=outp)
    return model

In [ ]:
input_vec = 24

with strategy.scope():
    reg = get_regressor()
    reg.compile(optimizer='adam', loss='mse')

In [ ]:
X, y = filtrate(reactivity_68, e1_68)
reg.fit(X, y, epochs=3)

In [ ]:
t1_pred = reg.predict(ts_features)
t1_pred.shape

In [ ]:
X, y = filtrate(deg_Mg_pH10_68, e2_68)
reg.fit(X, y, epochs=3)

t2_pred = reg.predict(ts_features)
t2_pred.shape

In [ ]:
X, y = filtrate(deg_pH10_68, e3_68)
reg.fit(X, y, epochs=3)

t3_pred = reg.predict(ts_features)
t3_pred.shape

In [ ]:
X, y = filtrate(deg_Mg_50C_68, e4_68)
reg.fit(X, y, epochs=3)

t4_pred = reg.predict(ts_features)
t4_pred.shape

In [ ]:
X, y = filtrate(deg_50C_68, e5_68)
reg.fit(X, y, epochs=3)

t5_pred = reg.predict(ts_features)
t5_pred.shape

# File processing

In [ ]:
ids = []
for iden in id_col_107:
    for i in range(107):
        ids.append(iden + '_' + str(i))
for iden in id_col_130:
    for i in range(130):
        ids.append(iden + '_' + str(i))
len(ids)

In [ ]:
final_df = pd.DataFrame({
    'id_seqpos': ids, 
    'reactivity': np.squeeze(t1_pred), 
    'deg_Mg_pH10': np.squeeze(t2_pred),
    'deg_pH10': np.squeeze(t3_pred),
    'deg_Mg_50C': np.squeeze(t4_pred),
    'deg_50C': np.squeeze(t5_pred)})
final_df.head()

In [ ]:
final_df.to_csv('submission.csv', index=False)

In [ ]:
final_df.reactivity.nunique()

# Continued Experiment

In [ ]:
exp_df

In [ ]:
ts_ft_df

In [ ]:
final_df

In [ ]:
final_df['feat'] = ts_ft_df.ts_feat
final_df

In [ ]:
final_df.drop('id_seqpos', axis=1, inplace=True)
final_df = final_df.drop_duplicates()
final_df

In [ ]:
df = exp_df.merge(final_df, on='feat', how='left')
df

In [ ]:
df.isna().sum()

In [ ]:
df = df.fillna(999)

In [ ]:
df.isna().sum()

In [ ]:
def corre(dataframe):
    temList = []
    for _, item in dataframe.iterrows():
        t = item[0]
        e = item[1]
        p = item[2]
        if p == 999:
            temList.append(t)
        elif t < p:
            temList.append(t + e)
        elif t > p:
            temList.append(t - e)
        else:
            temList.append(t)
    return temList

In [ ]:
df['t1_n'] = corre(df[['t1', 'e1', 'reactivity']])
df['t2_n'] = corre(df[['t2', 'e2', 'deg_Mg_pH10']])
df['t3_n'] = corre(df[['t3', 'e3', 'deg_pH10']])
df['t4_n'] = corre(df[['t4', 'e4', 'deg_Mg_50C']])
df['t5_n'] = corre(df[['t5', 'e5', 'deg_50C']])
df

In [ ]:
t1_n = np.array(df.t1_n)
t2_n = np.array(df.t2_n)
t3_n = np.array(df.t3_n)
t4_n = np.array(df.t4_n)
t5_n = np.array(df.t5_n)

## Experimental fit-predict

In [ ]:
X, y = filtrate(t1_n, e1_68)
reg.fit(X, y, epochs=3)

t1_pred2 = reg.predict(ts_features)
t1_pred.shape

In [ ]:
X, y = filtrate(t2_n, e2_68)
reg.fit(X, y, epochs=3)

t2_pred2 = reg.predict(ts_features)
t2_pred2.shape

In [ ]:
X, y = filtrate(t3_n, e3_68)
reg.fit(X, y, epochs=3)

t3_pred2 = reg.predict(ts_features)
t3_pred2.shape

In [ ]:
X, y = filtrate(t4_n, e4_68)
reg.fit(X, y, epochs=3)

t4_pred2 = reg.predict(ts_features)
t4_pred2.shape

In [ ]:
X, y = filtrate(t5_n, e5_68)
reg.fit(X, y, epochs=3)

t5_pred2 = reg.predict(ts_features)
t5_pred2.shape

## Experimental File-process

In [ ]:
sub2_df = pd.DataFrame({
    'id_seqpos': ids, 
    'reactivity': np.squeeze(t1_pred2), 
    'deg_Mg_pH10': np.squeeze(t2_pred2),
    'deg_pH10': np.squeeze(t3_pred2),
    'deg_Mg_50C': np.squeeze(t4_pred2),
    'deg_50C': np.squeeze(t5_pred2)})
sub2_df.head()

In [ ]:
sub2_df.to_csv('submission_exp.csv', index=False)

In [ ]:
def err(

# XGB Regressor

In [ ]:
def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

In [ ]:
regress = XGBRegressor()

In [ ]:
params = {
    'n_estimators': [3, 5, 10, 20],
    'max_dapth': [3, 5, 10, 20],
    'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1],
    'booster': ['gbtree', 'gblinear', 'dart'],
    'gamma': [0.00001, 0.0001, 0.0005, 0.001, 0.005, 0.008, 0.01, 0.1], #Minimum LR required for the split in a tree
    #'score': [rmse]
}

In [ ]:
gs_cv = GridSearchCV(estimator=regress, param_grid=params)

In [ ]:
gs_cv.fit(X, y)